In [36]:
import os
import pandas as pd
import pytesseract
from bs4 import BeautifulSoup

In [37]:
# Define the path to your data directory
data_dir = 'G:/data/data'

In [38]:
import os
import re
# os.chdir('G:/data/data')

In [39]:
os.listdir(data_dir)

['Balance Sheets', 'Cash Flow', 'Income Statement', 'Notes', 'Others']

In [40]:
# Initialize an empty list to hold the output data
output_data = []

# Define the Tesseract executable path if needed (depends on your installation)
# For example: pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to extract text from HTML content
def extract_text_from_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        soup = BeautifulSoup(content, 'html.parser')
        text = soup.get_text()
        return text
def filter_alphabetic(text):
    lines = text.split('\n')
    filtered_lines = [' '.join(re.findall(r'[a-zA-Z]+', line)) for line in lines]
    return '\n'.join(filtered_lines)

# Traverse the data directory
for folder_name in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder_name)
    if os.path.isdir(folder_path):
        for html_file in os.listdir(folder_path):
            if html_file.endswith('.html'):
                file_path = os.path.join(folder_path, html_file)
                text = extract_text_from_html(file_path)
                filtered_text = filter_alphabetic(text)
                # Append the data to the output list
                output_data.append({'docno': html_file, 'Text': filtered_text, 'Doc_target': folder_name})

# Create a DataFrame from the output data
df = pd.DataFrame(output_data)

# Save the DataFrame to a CSV file
df.to_csv('output.csv', index=False)


In [41]:
df = pd.read_csv('output.csv',low_memory=False)

In [42]:
k.head()

,docno,Text,Doc_target
0,18320959_3.html,\n\n\n\n\nin lacs\nin lacs\n\n\nStandalone\nCo...,Balance Sheets
1,18391125_2.html,\n\n\n\n\nConsolidated\nStandalone\n\nParticul...,Balance Sheets
2,18442877_5.html,\n\n\n\n\nStandalone\nConsolidated\n\n\nAudite...,Balance Sheets
3,18445487_2.html,\n\n\n\nGUJARAT NARMADA VALLEY FERTILIZERS CHE...,Balance Sheets
4,18445494_3.html,\n\n\n\n\n\nStandalone\nConsolidated\n\n\n\n\n...,Balance Sheets


In [43]:
print(k.loc[6,'Text'])





Particulars
As at
As ai


Audited
Audited

ASSETS



Non Current Assets



Property Plant and Equipment



Capital Work in progress



Investment Property



Other Intangible Assets



Financial Assets



lnve tiT erMs



Other Financial Assets



Loans



Other Non Current Assets



Total Non Current Assets



Current Assets



Inventories



Financial Assets



Investments



Trade Receivables



Service Concession Receivable



Cash ana Cash Equivalents



Bank Balance other than Cash and Cash Equivalents



above



Othet Financial Assets



Loans



Other Current Assets



Total Current Assets



Noo Current Assets Held for sale and Discontinued



Operations



Total Assets before Regulatory Assets



Regulatory deferral account debit balances and related



deferred tax balances



Total Assets



Equity and Liabilities



Equity



Equity Share Capital



Other Equily



Total Equity



LIABILITIES



Non Current Liabilities



Financial Liabilities



Sorrowings



Financi

In [44]:
k['Doc_target'].value_counts()

Doc_target
Others              1224
Notes                690
Income Statement     305
Balance Sheets       270
Cash Flow             36
Name: count, dtype: int64

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
from sentence_transformers import SentenceTransformer

In [47]:
# Initialize the BERT model for sentence embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings using BERT model
df['BERT_embeddings'] = df['Text'].apply(lambda x: model.encode(x))

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate TF-IDF embeddings
tfidf_embeddings = tfidf_vectorizer.fit_transform(df['Text']).toarray()

# Add TF-IDF embeddings to DataFrame
df['TFIDF_embeddings'] = list(tfidf_embeddings)

# Save the DataFrame to a CSV file
df.to_csv('output_with_embeddings.csv', index=False)

c:\Users\HP\Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [48]:
kk = pd.read_csv('output_with_embeddings.csv')
kk.head()

,docno,Text,Doc_target,BERT_embeddings,TFIDF_embeddings
0,18320959_3.html,\n\n\n\n\nin lacs\nin lacs\n\n\nStandalone\nCo...,Balance Sheets,[ 3.18422541e-02 -8.92421678e-02 -5.46203032e-...,[0. 0. 0. ... 0. 0. 0.]
1,18391125_2.html,\n\n\n\n\nConsolidated\nStandalone\n\nParticul...,Balance Sheets,[ 2.65763588e-02 -7.69121349e-02 -6.96762428e-...,[0. 0. 0. ... 0. 0. 0.]
2,18442877_5.html,\n\n\n\n\nStandalone\nConsolidated\n\n\nAudite...,Balance Sheets,[ 5.35886828e-03 -4.43138964e-02 -4.27106731e-...,[0. 0. 0. ... 0. 0. 0.]
3,18445487_2.html,\n\n\n\nGUJARAT NARMADA VALLEY FERTILIZERS CHE...,Balance Sheets,[-6.37322618e-03 -2.75479704e-02 -6.68377727e-...,[0. 0. 0. ... 0. 0. 0.]
4,18445494_3.html,\n\n\n\n\n\nStandalone\nConsolidated\n\n\n\n\n...,Balance Sheets,[ 1.99885461e-02 -5.37648201e-02 -4.81042638e-...,[0. 0. 0. ... 0. 0. 0.]


In [52]:
kk.loc[6,'BERT_embeddings']
print(kk.loc[6,'TFIDF_embeddings'])

[0. 0. 0. ... 0. 0. 0.]


In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the DataFrame with embeddings
df = pd.read_csv('output_with_embeddings.csv')
df = df[['docno','Text','Doc_target','BERT_embeddings']]
# df
# Assuming embeddings are stored as strings, convert them back to lists
df['BERT_embeddings'] = df['BERT_embeddings'].apply(eval)

# Extract features and target
X = np.array(df['BERT_embeddings'].tolist())
y = df['Doc_target']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize and train a classifier (e.g., Random Forest)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Save the model if needed
import joblib
joblib.dump(classifier, 'document_classifier.pkl')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)

In [58]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np
import joblib

# Define the path to your data directory
data_dir = 'G:/data/data'

# Initialize an empty list to hold the output data
output_data = []

# Function to extract text from HTML content
def extract_text_from_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        soup = BeautifulSoup(content, 'html.parser')
        text = soup.get_text()
        return text

# Function to retain only alphabetic characters in each line
def filter_alphabetic(text):
    lines = text.split('\n')
    filtered_lines = [' '.join(re.findall(r'[a-zA-Z]+', line)) for line in lines]
    return '\n'.join(filtered_lines)

# Traverse the data directory
for folder_name in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder_name)
    if os.path.isdir(folder_path):
        for html_file in os.listdir(folder_path):
            if html_file.endswith('.html'):
                file_path = os.path.join(folder_path, html_file)
                text = extract_text_from_html(file_path)
                filtered_text = filter_alphabetic(text)
                # Append the data to the output list
                output_data.append({'docno': html_file, 'Text': filtered_text, 'Doc_target': folder_name})

# Create a DataFrame from the output data
df = pd.DataFrame(output_data)

# Initialize the BERT model for sentence embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings using BERT model
df['BERT_embeddings'] = df['Text'].apply(lambda x: model.encode(x))

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate TF-IDF embeddings
tfidf_embeddings = tfidf_vectorizer.fit_transform(df['Text']).toarray()

# Add TF-IDF embeddings to DataFrame
df['TFIDF_embeddings'] = list(tfidf_embeddings)

# Combine BERT and TF-IDF embeddings (concatenate them)
combined_embeddings = np.hstack((np.array(df['BERT_embeddings'].tolist()), tfidf_embeddings))

# Prepare target labels
y = df['Doc_target']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_embeddings, y_encoded, test_size=0.2, random_state=42)

# Initialize and train a classifier (e.g., Random Forest)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Save the model if needed
joblib.dump(classifier, 'document_classifier.pkl')

# Save the DataFrame with embeddings to a CSV file
df.to_csv('output_with_embeddings.csv', index=False)

c:\Users\HP\Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


                  precision    recall  f1-score   support

  Balance Sheets       1.00      0.92      0.96        51
       Cash Flow       1.00      0.83      0.91         6
Income Statement       1.00      0.88      0.94        66
           Notes       0.89      0.83      0.86       123
          Others       0.89      0.96      0.92       259

        accuracy                           0.91       505
       macro avg       0.96      0.88      0.92       505
    weighted avg       0.92      0.91      0.91       505



: 